# Completing actor data

In [1]:
from src.data.transform_data import* 
from src.models.movie_success_model import* 
from src.models.actor_success_model import*
from webscraping.webscraper import*

## 1) Load actor data from csv

In [2]:
actor_df = pd.read_csv('data/actor_data.csv', index_col=0)
actor_df

,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor age at movie release,Actor Score Index
Actor name,,,,,,
denzel_washington,1954-12-28,M,1.84,/m/0x67,26.0,10.000000
matt_damon,1970-10-08,M,1.78,/m/0cnvdq1,18.0,9.912690
tom_hanks,1956-07-09,M,1.83,/m/02p4q5p,27.0,9.741555
eddie_murphy,1961-04-03,M,1.75,/m/0x67,21.0,9.415060
tom_cruise,1962-07-03,M,1.70,/m/02ctzb,18.0,9.362255
...,...,...,...,...,...,...
j._kenneth_campbell,1947-07-22,M,NaN,NaN,43.0,0.714754
evelyn_keyes,1916-11-20,F,1.63,NaN,72.0,0.700949
james_dixon,NaN,M,NaN,NaN,NaN,0.700949


## 2) Set sample size

In [3]:
test_sample = actor_df
test_sample = test_sample.dropna(subset='Actor height')

## 3) Run the scraper

In [4]:
# spider = ActorScraper(test_sample)
# test_sample = spider.run_scraping()
# test_sample

In [5]:
test_sample.to_csv('data/actor_data_scraped.csv')

In [6]:
show_data = pd.read_csv('webscraping/actor_data_scraped.csv')

In [42]:
rankings = pd.read_csv('data/2024_QS_World_University_Rankings_csv', header=0)
rankings

,0,2024 QS World University Rankings,2023 QS World University Rankings,Institution Name,Location_code,Location,Classification,Focus,res,Age,...,IFScore,International Students,ISScore,International Research Network,IRNScore,Employment Outcomes,EOScore,Sustainability,SusScore,Overall
0,3,1,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5.0,...,56,88.2,128,94.3,58,100.0,4,95.2,51,100
1,4,2,2,University of Cambridge,UK,United Kingdom,L,FC,VH,5.0,...,64,95.8,85,99.9,7,100.0,6,97.3,33=,99.2
2,5,3,4,University of Oxford,UK,United Kingdom,L,FC,VH,5.0,...,110,98.2,60,100.0,1,100.0,3,97.8,26=,98.9
3,6,4,5,Harvard University,US,United States,L,FC,VH,5.0,...,210,66.8,223,100.0,5,100.0,1,96.7,39,98.3
4,7,5,3,Stanford University,US,United States,L,FC,VH,5.0,...,78,51.2,284,95.8,44,100.0,2,94.4,63,98.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,1495,1401+,1401+,University of Oradea,RO,Romania,L,FC,MD,4.0,...,701+,7.0,701+,1.8,701+,5.9,701+,2.2,701+,-
1493,1496,1401+,NaN,University of Stavanger,NO,Norway,NaN,CO,NaN,2.0,...,NaN,NaN,NaN,26.3,489,NaN,NaN,NaN,NaN,-
1494,1497,1401+,1201-1400,"University Politehnica of Timisoara, UPT",RO,Romania,M,FO,VH,5.0,...,701+,2.9,701+,1.3,701+,8.5,701+,1.0,701+,-
1495,1498,1401+,1201-1400,Western Washington University,US,United States,L,CO,HI,5.0,...,676,1.4,701+,1.0,701+,7.3,701+,NaN,NaN,-


In [43]:
from rapidfuzz import process

In [44]:
# Match the names found from webscraping to the names as they are registered in the QS Rankings

def match_universities(uni_name, qs_uni):
    if not isinstance(uni_name, str):  # Check if the value is NaN
        return None
    result = process.extractOne(uni_name, qs_uni)
    if result:
        match, score, _ = result  # Unpack match, score, and index
        return match if score > 88 else None  # Return match only if confidence > 80
    return None

qs_uni_names = rankings['Institution Name'].to_list()
show_data['Matched Uni'] = show_data['University'].apply(lambda x: match_universities(x, qs_uni_names))

In [45]:
show_data['University'] = show_data.apply(
    lambda row: row['Matched Uni'] if pd.notna(row['Matched Uni']) else row['University'], axis = 1
)

show_data['found_match'] = show_data.apply(lambda row: True if pd.notna(row['Matched Uni']) else False, axis=1)

show_data.drop(columns=['Matched Uni'], inplace=True)

In [46]:
# Replace all values containing 'State University of New York at Purchase' with 'Purchase College SUNY'
show_data.loc[
    show_data['University'].str.contains('Purchase', case=False, na=False),
    'University'
] = 'Purchase College SUNY'


In [47]:
# Replace all values containing 'High School' with 'None'
show_data.loc[
    show_data['University'].str.contains('high school', case=False, na=False),
    'University'
] = None

# Replace Academy Award by None

show_data.loc[
    show_data['University'].str.contains('Academy Award', case=False, na=False),
    'University'
] = None

In [48]:
show_data.loc[
    show_data['University'].str.contains('drama', case=False, na=False),
    'University'
] = 'Specialised Drama School'

show_data.loc[
    show_data['University'].str.contains('dramatic', case=False, na=False),
    'University'
] = 'Specialised Drama School'

show_data.loc[
    show_data['University'].str.contains('theatre', case=False, na=False),
    'University'
] = 'Specialised Drama School'

show_data.loc[
    show_data['University'].str.contains('theater', case=False, na=False),
    'University'
] = 'Specialised Drama School'

show_data.loc[
    show_data['University'].str.contains('acting', case=False, na=False),
    'University'
] = 'Specialised Acting School'

show_data.loc[
    show_data['University'].str.contains('film', case=False, na=False),
    'University'
] = 'Specialised Acting School'

show_data.loc[
    show_data['University'].str.contains('performing arts', case=False, na=False),
    'University'
] = 'Specialised Drama School'

In [49]:
show_data.loc[
    show_data['University'].str.contains('oxford', case=False, na=False),
    'University'
] = 'University of Oxford'

show_data.loc[
    show_data['University'].str.contains('college cambridge', case=False, na=False),
    'University'
] = 'University of Cambridge'

show_data.loc[
    show_data['University'].str.contains('trinity hall', case=False, na=False),
    'University'
] = 'University of Cambridge'

show_data.loc[
    show_data['University'].str.contains('University of California Los Angeles', case=False, na=False),
    'University'
] = "University of California, Los Angeles (UCLA)"

show_data.loc[
    show_data['University'].str.contains('UCLA', case=False, na=False),
    'University'
] = "University of California, Los Angeles (UCLA)"

show_data.loc[
    show_data['University'].str.contains('University of California Santa Barbara', case=False, na=False),
    'University'
] = "University of California, Santa Barbara (UCSB)"

In [50]:
show_data.loc[
    show_data['University'].str.contains('University College London', case=False, na=False),
    'University'
] = 'UCL'

show_data.loc[
    show_data['University'].str.contains('Kings College London', case=False, na=False),
    'University'
] = "King's College London"

show_data.loc[
    show_data['University'].str.contains('University of Toronto', case=False, na=False),
    'University'
] = 'University of Toronto'

show_data.loc[
    show_data['University'].str.contains('University of Georgia', case=False, na=False),
    'University'
] = 'The University of Georgia'

show_data.loc[
    show_data['University'].str.contains('Harvard', case=False, na=False),
    'University'
] = 'University of Harvard'

show_data.loc[
    show_data['University'].str.contains('University of Missouri', case=False, na=False),
    'University'
] = 'University of Missouri'

show_data.loc[
    show_data['University'].str.contains('University of North Carolina', case=False, na=False),
    'University'
] = 'University of North Carolina'

show_data.loc[
    show_data['University'].str.contains('San Diego State', case=False, na=False),
    'University'
] = 'San Diego State University'

show_data.loc[
    show_data['University'].str.contains('University of Texas', case=False, na=False),
    'University'
] = 'University of Texas'

show_data.loc[
    show_data['University'].str.contains('University of Alabama', case=False, na=False),
    'University'
] = 'University of Alabama'

show_data.loc[
    show_data['University'].str.contains('University of Michigan', case=False, na=False),
    'University'
] = 'University of Michigan-Ann Arbor'

show_data.loc[
    show_data['University'].str.contains('Texas AM', case=False, na=False),
    'University'
] = 'Texas A&M University'

show_data.loc[
    show_data['University'].str.contains('Fordham', case=False, na=False),
    'University'
] = 'Fordham University'

show_data.loc[
    show_data['University'].str.contains('Rutgers', case=False, na=False),
    'University'
] = 'Rutgers University–New Brunswick'

In [51]:
# Handle Music Schools

show_data.loc[
    show_data['University'].str.contains('Music', case=False, na=False),
    'University'
] = 'Specialised Music School'

show_data.loc[
    show_data['University'].str.contains('Conservatory', case=False, na=False),
    'University'
] = 'Specialised Music School'

show_data.loc[
    show_data['University'].str.contains('Conservatoire', case=False, na=False),
    'University'
] = 'Specialised Music School'

show_data.loc[
    show_data['University'].str.contains('Juilliard', case=False, na=False),
    'University'
] = 'Specialised Music School'

In [52]:
# Ballet schools

show_data.loc[
    show_data['University'].str.contains('ballet', case=False, na=False),
    'University'
] = 'Specialised Dance School'

show_data.loc[
    show_data['University'].str.contains('dance', case=False, na=False),
    'University'
] = 'Specialised Dance School'

In [53]:
# Handle Arts students

show_data.loc[
    show_data['University'].str.contains('art', case=False, na=False),
    'University'
] = 'Specialised Arts School'

show_data.loc[
    show_data['University'].str.contains('arts', case=False, na=False),
    'University'
] = 'Specialised Arts School'

In [54]:
# Match the names found from webscraping to the names as they are registered in the QS Rankings

def match_universities(uni_name, qs_uni):
    if not isinstance(uni_name, str):  # Check if the value is NaN
        return None
    result = process.extractOne(uni_name, qs_uni)
    if result:
        match, score, _ = result  # Unpack match, score, and index
        return match if score > 88 else None  # Return match only if confidence > 80
    return None

qs_uni_names = rankings['Institution Name'].to_list()
show_data['Matched Uni'] = show_data['University'].apply(lambda x: match_universities(x, qs_uni_names))

In [55]:
show_data['University'] = show_data.apply(
    lambda row: row['Matched Uni'] if pd.notna(row['Matched Uni']) else row['University'], axis = 1
)

show_data['found_match'] = show_data.apply(lambda row: True if pd.notna(row['Matched Uni']) else False, axis=1)

show_data.drop(columns=['Matched Uni'], inplace=True)

In [56]:
show_data.loc[
    show_data['University'].str.contains('vassar', case=False, na=False),
    'University'
] = 'Vassar College'

In [58]:
# Update 'University' where conditions are met
show_data.loc[
    (show_data['found_match'] == False) &  # Check if 'found_match' is False
    (~(show_data['University'] == None)) & # Check if empty
    (~show_data['University'].str.lower().str.startswith('specialised', na=False)) &  # Does NOT start with 'specialized'
    (~show_data['University'].str.lower().str.startswith('vassar', na=False)),  # Does NOT start with 'vassar'
    'University'
] = 'sub 1500 school'
